In [2]:
import bs4
import pandas as pd
import requests
import ast
import time
from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen
import pyodbc
import urllib.request as urllib_request
from urllib.request import urlopen




In [3]:
url = "http://api.willcode.tech/funcionarios/?USUARIO=USUARIO&SENHA=SENHA_SECRETA&ACAO=LISTAR-TODOS&SHOW_EMPTY"
response = requests.request("GET", url)

In [5]:
#Informando o agente
agente = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
headers = {'User-Agent': agente}

In [6]:
def ConsultaWeb(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        print(response.read())

    except HTTPError as e:
        print('excessao 1 -> dominio existe, pagina não')    
        print(e.status, e.reason)

    except URLError as e: 
        print('excessao 2 -> dominio não existe')
        print(e.reason)

In [7]:
def ConsultaWebB(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()

    except:
        pass

In [9]:
def ConsultaWebTO(url):
    try:
        req = requests.get(url, headers = headers, timeout = 1) 
        return req.content      

    except:
        pass

In [10]:
html = ConsultaWebB('https://www.camaraindaial.sc.gov.br/pg/proposicoes')
soup = bs4.BeautifulSoup(html, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="h-100">
 <head>
  <!--GENERAL-->
  <meta charset="utf-8"/>
  <meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport"/>
  <!--PAGE TYPE-->
  <meta content="website" property="og:type"/>
  <meta content="summary_large_image" property="twitter:card"/>
  <!--TITLE-->
  <title>
   Proposições   |  Câmara Municipal de Indaial
  </title>
  <meta content="Proposições" property="twitter:title"/>
  <meta content="Proposições" property="og:title"/>
  <!--SUBTITLE-->
  <meta content="" property="twitter:description"/>
  <meta content="" property="og:description"/>
  <!--DESCRIPTION-->
  <meta content="Conteúdo estático com links para o Legislador." property="description"/>
  <!--IMGS-->
  <meta content="https://www.camaraindaial.sc.gov.br/view/dist/imgs/metatag.jpg" property="twitter:image"/>
  <meta content="https://www.camaraindaial.sc.gov.br/view/dist/imgs/metatag.jpg" property="twitter:image:alt"/>
  <meta content="https://www.camaraindai

In [11]:
def captura_html_pagina(url):
    html = ConsultaWebB(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    return soup

In [12]:
#Agora sim iniciando o processo para criar o banco de dados com as informações que precisamos, as bibliotecas todas forma importadas desta forma continuemos com o trabalho



In [13]:
#Criando a função para chamar a página
def ConsultaWebB(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()
    except:
        pass

In [14]:
#Transformando a página em um objeto utilizável
def captura_html_pagina(url):
    html = ConsultaWebB(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    return soup

In [15]:
#Função para capturar as informações  da tabela da página

def Cabecalho(html):
    dt = html.find_all('dt')
    dd = html.find_all('dd')
    dic = {}
    for i in range (len(dt)):
        x = dt[i].get_text()
        y = dd[i].get_text()
        dic[x] = y
    return dic 

In [16]:
#Função para obter os dados em forma de dicionário

def Conteudo(proposicao, ano):
    url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(proposicao)+'&aaProposicao='+str(ano)
    html = captura_html_pagina(url)
    dic = Cabecalho(html)
    dic['Proposição'] = proposicao
    dic['Ano'] = ano
    dic['Texto'] = html.p.get_text()
    return dic

In [17]:
#Função para automatização do processo de coletar dados utilizando o laço while

def TabelaResultados (iniciar_em, quantidade, ano, erros_admissiveis, segundos_espera):
    ultima_consulta = iniciar_em + quantidade - 1
    
    erros = 0
    
    i = 1
    lista = []
    while iniciar_em <= ultima_consulta and erros <= erros_admissiveis:
        try:
            x = Conteudo(iniciar_em, ano)
            lista = lista + [Conteudo(iniciar_em, ano)]
        except:
            errors += 1
            pass
        time.sleep(segundos_espera)
        iniciar_em+=1
        i+=1
        
    return pd.DataFrame(lista)

In [18]:
teste = TabelaResultados(500,3,2021,2,0.2)
teste

,Reunião,Deliberação,Situação,Assunto,Autor,Proposição,Ano,Texto
0,08/04/2021,08/04/2021,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Flávio Augusto Ferri Molinari.,500,2021,"O vereador que está subscreve, no uso das atri..."
1,08/04/2021,08/04/2021,Proposição Aprovada,"Pontes, Bueiros, boca de lobo",Vereador Flávio Augusto Ferri Molinari.,501,2021,"O vereador que está subscreve, no uso das atri..."
2,08/04/2021,08/04/2021,Proposição Aprovada,Manutenção de via pública,Vereador Fernanda dos Santos.,502,2021,"A vereadora que esta subscreve, no uso das atr..."


In [19]:
#Estabelecendo conexão com o banco de dados criado no SQL server, Lembrar de alterar o server

conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'JEZANDRE\MSSQLSERVER2', 
                      database = 'ScrapJovi2')

query = '''
    select 
        * 
    from Proposicoes
'''
sql_query = pd.read_sql_query(query,conn)
sql_query

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.,1,1996,construção de Escola de 1º grau no Bairro Nova...
1,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.,2,1996,construção de Escola nas imediações dos Loteam...
2,1996-02-22,1996-02-22,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Henrique Fritz.,3,1996,alargamento da Rua ID 90 ...
3,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.,4,1996,prolongamento da rede d'água na Rua Reinhold S...
4,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.,5,1996,Prolongamento de rede d'água na Rua Lorenz até...
...,...,...,...,...,...,...,...,...
3297,1900-01-01,1900-01-01,,,,3277,1996,Usamos cookies e tecnologias semelhantes para ...
3298,1900-01-01,1900-01-01,,,,3295,1996,Usamos cookies e tecnologias semelhantes para ...
3299,1900-01-01,1900-01-01,,,,3296,1996,Usamos cookies e tecnologias semelhantes para ...
3300,1900-01-01,1900-01-01,,,,3298,1996,Usamos cookies e tecnologias semelhantes para ...


In [20]:
#Criar uma base, assim não corre o risco de uma informação em branco avacalhar a inserção dos dados na tabela
base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
base

,Reunião,Deliberação,Situação,Assunto,Autor,Proposição,Ano,Texto


In [21]:
#começando a inserir dados na base
base = base.append(teste)
base


C:\Users\jezan\AppData\Local\Temp\ipykernel_6792\331753047.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append(teste)


,Reunião,Deliberação,Situação,Assunto,Autor,Proposição,Ano,Texto
0,08/04/2021,08/04/2021,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Flávio Augusto Ferri Molinari.,500,2021,"O vereador que está subscreve, no uso das atri..."
1,08/04/2021,08/04/2021,Proposição Aprovada,"Pontes, Bueiros, boca de lobo",Vereador Flávio Augusto Ferri Molinari.,501,2021,"O vereador que está subscreve, no uso das atri..."
2,08/04/2021,08/04/2021,Proposição Aprovada,Manutenção de via pública,Vereador Fernanda dos Santos.,502,2021,"A vereadora que esta subscreve, no uso das atr..."


In [22]:
#Comando para substituir valores nulos
base.fillna('')

,Reunião,Deliberação,Situação,Assunto,Autor,Proposição,Ano,Texto
0,08/04/2021,08/04/2021,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Flávio Augusto Ferri Molinari.,500,2021,"O vereador que está subscreve, no uso das atri..."
1,08/04/2021,08/04/2021,Proposição Aprovada,"Pontes, Bueiros, boca de lobo",Vereador Flávio Augusto Ferri Molinari.,501,2021,"O vereador que está subscreve, no uso das atri..."
2,08/04/2021,08/04/2021,Proposição Aprovada,Manutenção de via pública,Vereador Fernanda dos Santos.,502,2021,"A vereadora que esta subscreve, no uso das atr..."


In [23]:
#inserindo dados na tabela do SQL

base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
teste = base.append(teste).fillna('')

conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'JEZANDRE\MSSQLSERVER2', 
                      database = 'ScrapJovi2')
cursor = conn.cursor()

for index, row in teste.iterrows():
    cursor.execute('''
        Set Language 'Português'
    
        INSERT INTO Proposicoes (
            DataReuniao,
            DataDeliberacao,
            Situacao,
            Assunto,
            Autor,
            Proposicao,
            Ano,
            Texto
        ) 
        values(?,?,?,?,?,?,?,?)''',
        row['Reunião'], 
        row['Deliberação'], 
        row['Situação'], 
        row['Assunto'], 
        row['Autor'], 
        row['Proposição'], 
        row['Ano'], 
        row['Texto']
    )
    
conn.commit()
cursor.close()  
    


C:\Users\jezan\AppData\Local\Temp\ipykernel_6792\2467987277.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teste = base.append(teste).fillna('')


In [24]:
#Agora testando para ver se os dados realmente forma inseridos no banco SQL

conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'JEZANDRE\MSSQLSERVER2', 
                      database = 'ScrapJovi2')
query = '''
    select 
        * 
    from Proposicoes
'''
sql_query = pd.read_sql_query(query,conn)
sql_query

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.,1,1996,construção de Escola de 1º grau no Bairro Nova...
1,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.,2,1996,construção de Escola nas imediações dos Loteam...
2,1996-02-22,1996-02-22,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Henrique Fritz.,3,1996,alargamento da Rua ID 90 ...
3,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.,4,1996,prolongamento da rede d'água na Rua Reinhold S...
4,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.,5,1996,Prolongamento de rede d'água na Rua Lorenz até...
...,...,...,...,...,...,...,...,...
3300,1900-01-01,1900-01-01,,,,3277,1996,Usamos cookies e tecnologias semelhantes para ...
3301,1900-01-01,1900-01-01,,,,3295,1996,Usamos cookies e tecnologias semelhantes para ...
3302,1900-01-01,1900-01-01,,,,3296,1996,Usamos cookies e tecnologias semelhantes para ...
3303,1900-01-01,1900-01-01,,,,3298,1996,Usamos cookies e tecnologias semelhantes para ...


In [25]:
#transformando os passos acima em uma função pra deixar nossa vida mais fácil e feliz

def SQLSelect(query):
    conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'JEZANDRE\MSSQLSERVER2', 
                      database = 'ScrapJovi2')
    out = pd.read_sql_query(query, conn)
    return out

In [26]:
SQLSelect('''select * from Proposicoes''')

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.,1,1996,construção de Escola de 1º grau no Bairro Nova...
1,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.,2,1996,construção de Escola nas imediações dos Loteam...
2,1996-02-22,1996-02-22,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Henrique Fritz.,3,1996,alargamento da Rua ID 90 ...
3,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.,4,1996,prolongamento da rede d'água na Rua Reinhold S...
4,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.,5,1996,Prolongamento de rede d'água na Rua Lorenz até...
...,...,...,...,...,...,...,...,...
3300,1900-01-01,1900-01-01,,,,3277,1996,Usamos cookies e tecnologias semelhantes para ...
3301,1900-01-01,1900-01-01,,,,3295,1996,Usamos cookies e tecnologias semelhantes para ...
3302,1900-01-01,1900-01-01,,,,3296,1996,Usamos cookies e tecnologias semelhantes para ...
3303,1900-01-01,1900-01-01,,,,3298,1996,Usamos cookies e tecnologias semelhantes para ...


In [27]:
SQLSelect('''select maior = max(Proposicao) from Proposicoes''')

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,maior
0,3302


In [28]:
#Agora vamos inserir os dados na tabela proposições:
def SQLInsertProposicoes(TabelaProposicoes):
    
    base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
    TabelaProposicoes = base.append(TabelaProposicoes).fillna('')

    conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'JEZANDRE\MSSQLSERVER2', 
                      database = 'ScrapJovi2')

    cursor = conn.cursor()

    for index, row in TabelaProposicoes.iterrows():

        cursor.execute('''
            Set Language 'Português'

            INSERT INTO Proposicoes (
                DataReuniao,
                DataDeliberacao,
                Situacao,
                Assunto,
                Autor,
                Proposicao,
                Ano,
                Texto
            ) 
            values(?,?,?,?,?,?,?,?)''', 

            row['Reunião'], 
            row['Deliberação'], 
            row['Situação'], 
            row['Assunto'], 
            row['Autor'], 
            row['Proposição'], 
            row['Ano'], 
            row['Texto']

        )

    conn.commit()
    cursor.close()

In [29]:
SQLInsertProposicoes(teste)

C:\Users\jezan\AppData\Local\Temp\ipykernel_6792\118395409.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  TabelaProposicoes = base.append(TabelaProposicoes).fillna('')


In [30]:
SQLSelect('''select * from Proposicoes''')

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.,1,1996,construção de Escola de 1º grau no Bairro Nova...
1,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.,2,1996,construção de Escola nas imediações dos Loteam...
2,1996-02-22,1996-02-22,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Henrique Fritz.,3,1996,alargamento da Rua ID 90 ...
3,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.,4,1996,prolongamento da rede d'água na Rua Reinhold S...
4,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.,5,1996,Prolongamento de rede d'água na Rua Lorenz até...
...,...,...,...,...,...,...,...,...
3303,1900-01-01,1900-01-01,,,,3296,1996,Usamos cookies e tecnologias semelhantes para ...
3304,2021-04-08,2021-04-08,Proposição Aprovada,"Pontes, Bueiros, boca de lobo",Vereador Flávio Augusto Ferri Molinari.,501,2021,"O vereador que está subscreve, no uso das atri..."
3305,2021-04-08,2021-04-08,Proposição Aprovada,Manutenção de via pública,Vereador Fernanda dos Santos.,502,2021,"A vereadora que esta subscreve, no uso das atr..."
3306,1900-01-01,1900-01-01,,,,3298,1996,Usamos cookies e tecnologias semelhantes para ...


In [31]:
#Utilizando Truncate para limpar a tabela antes de inserir novos dados:
def SQLTruncate(NomeTabela):
    
    conn = pyodbc.connect('Trusted_Connection=yes', 
                    driver = '{ODBC Driver 17 for SQL Server}',
                    server = 'JEZANDRE\MSSQLSERVER2', 
                    database = 'ScrapJovi2')
    cursor = conn.cursor()
    
    cursor.execute(f'''
                    TRUNCATE TABLE {NomeTabela}
                    ''')
    conn.commit()
    cursor.close()

In [32]:
#Testando a função truncate
SQLTruncate('Proposicoes')
SQLSelect('''select * from Proposicoes''')

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [33]:
#Logo depois de criar essas funções testar podemos agora automatizar a inserção de dados na nossa tabela assim poderemos em fim alimentar e criar nosso banco de dados

In [34]:
#Iniciando o processo de inserção

proposicao = 202
ano = 2021
dados = Conteudo(proposicao, ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)


C:\Users\jezan\AppData\Local\Temp\ipykernel_6792\118395409.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  TabelaProposicoes = base.append(TabelaProposicoes).fillna('')


In [35]:
#visulaizar a inserção
SQLSelect(f'select * from Proposicoes where Proposicao = {proposicao} and Ano = {ano}')

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,2021-02-11,2021-02-11,Proposição Aprovada,"Pontes, Bueiros, boca de lobo",Vereador Valentim Blasius.,202,2021,"O Vereador abaixo firmado requer, após ouvido ..."


In [36]:
ano = 2021
dados = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
dados

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Proposicao
0,202


In [37]:
# acessando a coluna
dados['Proposicao']

0    202
Name: Proposicao, dtype: int64

In [38]:
# acessando a coluna e linha
dados['Proposicao'].loc[0]

202

In [39]:
#Agora inseridndo dados incrementais de forma a sempre pegar o o maior numero de proporsicao gravado e inserir o próximo
ano = 2021

dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
ultima_proposicao = dados_ano['Proposicao'].loc[0]

proxima_proposicao = int(ultima_proposicao)+1

dados = Conteudo(proxima_proposicao, ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)


C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\jezan\AppData\Local\Temp\ipykernel_6792\118395409.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  TabelaProposicoes = base.append(TabelaProposicoes).fillna('')


In [40]:
#Visualizando dados
SQLSelect('select * from Proposicoes')

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,2021-02-11,2021-02-11,Proposição Aprovada,"Pontes, Bueiros, boca de lobo",Vereador Valentim Blasius.,202,2021,"O Vereador abaixo firmado requer, após ouvido ..."
1,2021-02-11,2021-02-11,Proposição Aprovada,Iluminação Pública e Rede de Energia Elétrica,Vereador Valentim Blasius.,203,2021,"O Vereador abaixo firmado requer, após ouvido ..."


In [41]:
#Limpando e visualizando

SQLTruncate('Proposicoes')
SQLSelect('select * from Proposicoes')

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [42]:
#Adicionando condivional se caso o campo de proposicao vier  vazio

ano = 1999

dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
ultima_proposicao = dados_ano['Proposicao'].loc[0]

if ultima_proposicao == None:
    proxima_proposicao = 1
else:
    proxima_proposicao = int(ultima_proposicao)+1

dados = Conteudo(proxima_proposicao, ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)

SQLSelect('select * from Proposicoes')

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\jezan\AppData\Local\Temp\ipykernel_6792\118395409.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  TabelaProposicoes = base.append(TabelaProposicoes).fillna('')
C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,1999-03-02,1999-03-02,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Remir José de Faveri.,1,1999,"Limpeza e patrolamento do Beco São Sebastião, ..."


In [43]:
#Definindo função para inserção de dados baseados no ano
def InsereProximaProposicao(ano):

    dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
    ultima_proposicao = dados_ano['Proposicao'].loc[0]
    
    if ultima_proposicao == None:
        proxima_proposicao = 1

    else:
        proxima_proposicao = int(ultima_proposicao) + 1 


    dados = Conteudo(proxima_proposicao,ano)
    tabela = pd.DataFrame([dados])
    SQLInsertProposicoes(tabela)

In [44]:
"""Observação os dados de 1996 até 1998 parecem não estar mais disponíveis então não consegui inserir esses dados. 
Na verdade a besta aqui instalou o SQLserver em ingles e o formato enviado está em portugues. Então na hora de enviar a data
Estava dando erro, procurei uma forma pra resolver isso e utilizei um método que set o idioma quando vai executar a query
e resolvi o problema, agora podemos continuar o processo espero que sem problemas agora
"""
InsereProximaProposicao(1996)
SQLSelect('select * from Proposicoes')

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\jezan\AppData\Local\Temp\ipykernel_6792\118395409.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  TabelaProposicoes = base.append(TabelaProposicoes).fillna('')
C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,1999-03-02,1999-03-02,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Remir José de Faveri.,1,1999,"Limpeza e patrolamento do Beco São Sebastião, ..."
1,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.,1,1996,construção de Escola de 1º grau no Bairro Nova...


In [45]:
#Limpando e visualizando

SQLTruncate('Proposicoes')
SQLSelect('select * from Proposicoes')

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [46]:
ano = 1996
quantidade = 99999
erros_admissiveis = 2
segundos_espera = 0.1

erros = 0

i = 1
lista = []

while erros <= erros_admissiveis:
    
    try:
        InsereProximaProposicao(ano)
        if data_ano == None:
            erros += 1
        else:
            pass
    except:
        erros += 1
        pass
    time.sleep(segundos_espera)
    
    i += 1

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\jezan\AppData\Local\Temp\ipykernel_6792\118395409.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  TabelaProposicoes = base.append(TabelaProposicoes).fillna('')
C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\jezan\AppData\Local\Temp\ipykernel_6792\118395409.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [47]:
def ChecaErro(url):
    html = captura_html_pagina(url)
    Erro = html.find_all(class_="col-10")
    if(Erro == None):
        return True
    else: 
        return False

In [53]:
def BuscaGravaDadosAno(ano, quantidade = 10, erros_admissiveis = 2, segundos_espera = 0):

    # erros
    erros = 0
    i = 1
    lista = []
    cont = 1

    while erros <= erros_admissiveis and cont <= quantidade:

        try:
            cont += 1
            url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(proposicao)+'&aaProposicao='+str(ano)
            if(ChecaErro(url)) == False:
                 InsereProximaProposicao(ano)
            else:
                erros += 1
                pass
                
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis
        i +=1

In [49]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select *from Proposicoes')

C:\Users\jezan\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [50]:
ano_fim = 1996
ano_ini = 1996
anos = ano_fim - ano_ini + 1
for i in range(anos):
    print(i+ano_ini)

1996


In [51]:
list(range(1996,1998+1))

[1996, 1997, 1998]

In [ ]:
ano_inicial = 2006
ano_final = 2021

for i in list(range(ano_inicial, ano_final+1)):
    print('Iniciando gravação dos dados do ano: ',i)
    try:
        BuscaGravaDadosAno(i, quantidade = 3000)
    except:
        pass
print('Inserção finalizada 😁')       